In [1]:
from decouple import config, AutoConfig

In [2]:
config = AutoConfig(search_path='.')

In [3]:
assert config("ALPHA_VANTAGE_API_KEY", default=None, cast=str) is not None

In [4]:
ALPHA_VANTAGE_API_KEY = config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

In [5]:
ALPHA_VANTAGE_API_KEY

'JTTWOF0TPR4M84BG'

In [6]:
import requests

params = {
    "api_key" : ALPHA_VANTAGE_API_KEY,
    "ticker": "AAPL",
    "function": "TIME_SERIES_INTRADAY",
    "interval": "5min",
}

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval={interval}&apikey={api_key}'.format(
    **params
)

r = requests.get(url)
data = r.json()

In [17]:
dataset_key = [x for x in list(data.keys()) if not x == "Meta Data"][0]

In [19]:
results = data[dataset_key]

In [25]:
timestamp_str = list(results.keys())[0]
timestamp_str

'2024-11-29 16:55:00'

In [38]:
result_val = results.get(timestamp_str)

In [41]:
import pytz
from datetime import datetime
from decimal import Decimal

def transform_alpha_vantage_result(timestamp_str, result):
    timestamp_format = "%Y-%m-%d %H:%M:%S"
    eastern = pytz.timezone("US/Eastern")
    utc = pytz.utc

    utc_timestamp = eastern.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc)

    #unix_timestamp = result.get('t') / 1000.0
    #utc_timestamp = datetime.fromtimestamp(unix_timestamp, tz=pytz.timezone('UTC'))

    
    return {
        'open_price': Decimal(result['1. open']),
        'close_price': Decimal(result['4. close']),
        'high_price': Decimal(result['2. high']),
        'low_price': Decimal(result['3. low']),
        'number_of_trades': None,
        'volume': int(result['5. volume']),
        'volume_weighted_average': None,
        'time': utc_timestamp,
    }

In [43]:
transform_alpha_vantage_result(timestamp_str, result_val)

{'open_price': Decimal('237.3700'),
 'close_price': Decimal('237.4450'),
 'high_price': Decimal('237.4900'),
 'low_price': Decimal('237.3000'),
 'number_of_trades': None,
 'volume': 6979,
 'volume_weighted_average': None,
 'time': datetime.datetime(2024, 11, 29, 21, 55, tzinfo=<UTC>)}

In [44]:
for timestamp_str in results.keys():
    new_data = transform_alpha_vantage_result(timestamp_str, results.get(timestamp_str))
    print(new_data)

{'open_price': Decimal('237.3700'), 'close_price': Decimal('237.4450'), 'high_price': Decimal('237.4900'), 'low_price': Decimal('237.3000'), 'number_of_trades': None, 'volume': 6979, 'volume_weighted_average': None, 'time': datetime.datetime(2024, 11, 29, 21, 55, tzinfo=<UTC>)}
{'open_price': Decimal('237.3850'), 'close_price': Decimal('237.3250'), 'high_price': Decimal('237.4000'), 'low_price': Decimal('237.3000'), 'number_of_trades': None, 'volume': 7640, 'volume_weighted_average': None, 'time': datetime.datetime(2024, 11, 29, 21, 50, tzinfo=<UTC>)}
{'open_price': Decimal('237.4994'), 'close_price': Decimal('237.4000'), 'high_price': Decimal('237.5000'), 'low_price': Decimal('237.3850'), 'number_of_trades': None, 'volume': 4280, 'volume_weighted_average': None, 'time': datetime.datetime(2024, 11, 29, 21, 45, tzinfo=<UTC>)}
{'open_price': Decimal('237.4100'), 'close_price': Decimal('237.4800'), 'high_price': Decimal('237.5000'), 'low_price': Decimal('237.4100'), 'number_of_trades': No